In [14]:
#packages

from urllib.request import urlretrieve

import os
import pandas as pd

%pip install bokeh --quiet

from bokeh.layouts import column, layout
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import HoverTool, Div, DatetimeRangeSlider, CustomJS
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.io import output_notebook

from datetime import datetime, timedelta, date

Note: you may need to restart the kernel to use updated packages.


In [15]:
#constants
path_inputs = "Inputs\\"

date_first = datetime(2024,1,1,0,0)

width_ = int(1000*7.4375/5.58)
height_ = int(1000*7.4375/5.58 /3)

line_width = 0.5

labels =            {'CAPITL':"Zone F: Capital", 
                      'CENTRL':"Zone C: Central", 
                      'DUNWOD':"Zone I: Dunwoodie", 
                      'GENESE':"Zone B: Genesee", 
                      'H Q':"Hydro-Quebec", 
                      'HUD VL':"Zone G: Hudson Valley", 
                      'LONGIL':"Zone K: Long Island",
                      'MHK VL':"Zone E: Mohawk Valley", 
                      'MILLWD':"Zone H: Millwood", 
                      'N.Y.C.':"Zone J: NYC", 
                      'NORTH':"Zone D: North", 
                      'NPX':"NE", 
                      'O H':"IESO", 
                      'PJM':"PJM", 
                      'WEST':"Zone A: West",
                      'CENTRAL EAST - VC':"?", 
                      'DYSINGER EAST':"?", 
                      'MOSES SOUTH':"?",
                      'SCH - HQ - NY':"HP Chateauguay", 
                      'SCH - HQ_CEDARS':"HQ Cedars", 
                      'SCH - HQ_IMPORT_EXPORT':"HP Chateauguay",
                      'SCH - NE - NY':"ISO NE", 
                      'SCH - NPX_1385':"NPX 1385 Northport", 
                      'SCH - NPX_CSC':"NPX Cross Sound Cable",
                      'SCH - OH - NY':"?", 
                      'SCH - PJ - NY':"?", 
                      'SCH - PJM_HTP':"?",
                      'SCH - PJM_NEPTUNE':"PJM Neptune", 
                      'SCH - PJM_VFT':"?", 
                      'SPR/DUN-SOUTH':"?",
                      'TOTAL EAST':"?", 
                      'UPNY CONED':"?", 
                      'WEST CENTRAL':"?"}

color_zone =         {'CAPITL':"#101010", 
                      'CENTRL':"#202020", 
                      'DUNWOD':"#303030", 
                      'GENESE':"#404040", 
                      'H Q':"blue", 
                      'HUD VL':"#505050", 
                      'LONGIL':"#606060",
                      'MHK VL':"#707070", 
                      'MILLWD':"#808080", 
                      'N.Y.C.':"red", 
                      'NORTH':"#909090", 
                      'NPX':"green", 
                      'O H':"#a0a0a0", 
                      'PJM':"purple", 
                      'WEST':"#c0c0c0"}

color_interface =    {'CENTRAL EAST - VC':"lightgrey", 
                      'DYSINGER EAST':"lightgrey",  
                      'MOSES SOUTH':"#33ba37",
                      'SCH - HQ - NY':"#4272E7", 
                      'SCH - HQ_CEDARS':"#81A1EF", 
                      'SCH - HQ_IMPORT_EXPORT':"#81A1EF",
                      'SCH - NE - NY':"#50c454", 
                      'SCH - NPX_1385':"#8ad88d", 
                      'SCH - NPX_CSC':"#c5ecc6",
                      'SCH - OH - NY':"lightgrey",  
                      'SCH - PJ - NY':"#bf8fce", 
                      'SCH - PJM_HTP':"lightgrey", 
                      'SCH - PJM_NEPTUNE':"#9f57b5", 
                      'SCH - PJM_VFT':"lightgrey",  
                      'SPR/DUN-SOUTH':"lightgrey", 
                      'TOTAL EAST':"lightgrey",  
                      'UPNY CONED':"lightgrey",  
                      'WEST CENTRAL':"lightgrey"}

#### Download from NYISO

In [16]:
def get_dates_between(start_date, end_date):
    dates = []
    current_date = start_date + timedelta(days=1)
    while current_date < end_date:
        _year = str(current_date.year)
        _month = str(current_date.month)
        if len(_month) == 1:
            _month = "0" + _month
        _day = str(current_date.day)
        if len(_day) == 1:
            _day = "0" + _day
            DATE = _year + _month + _day
        try:
            dates.append(DATE)
        except:
            pass
        current_date += timedelta(days=1)
    return dates

In [17]:
#get today's date in file format

_year = str(datetime.today().year)
_month = str(datetime.today().month)
if len(_month) == 1:
    _month = "0" + _month
_day = str(datetime.today().day)
if len(_day) == 1:
    _day = "0" + _day
today = _year + _month + _day
dt_today = date(int(_year),int(_month),int(_day))
today

'20260204'

In [18]:
#realtime file setup
url_realtime = "https://mis.nyiso.com/public/csv/realtime/"

file_realtime = "realtime_zone.csv"
path_realtime = "Inputs\\realtime\\"
list_realtime = os.listdir(path_realtime)
df_dates_realtime = pd.DataFrame(list_realtime)
df_dates_realtime["Year"] = df_dates_realtime[0].str[:4]
df_dates_realtime["Month"] = df_dates_realtime[0].str[4:6]
df_dates_realtime["Day"] = df_dates_realtime[0].str[6:8]
date_last_realtime = df_dates_realtime.sort_values(["Year","Month","Day"], ascending=[False, False, False]).reset_index().iloc[0,1][:8]
dt_last_realtime = date(int(date_last_realtime[:4]),int(date_last_realtime[4:6]),int(date_last_realtime[6:8]))
list_dates_realtime = get_dates_between(dt_last_realtime, dt_today)
list_dates_realtime

['20260201', '20260202', '20260203']

In [19]:
#dayahead file setup
url_dayahead = "https://mis.nyiso.com/public/csv/damlbmp/"
file_dayahead = "damlbmp_zone.csv"
path_dayahead = "Inputs\\dayahead\\"
list_dayahead = os.listdir(path_dayahead)
df_dates_dayahead = pd.DataFrame(list_dayahead)
df_dates_dayahead["Year"] = df_dates_dayahead[0].str[:4]
df_dates_dayahead["Month"] = df_dates_dayahead[0].str[4:6]
df_dates_dayahead["Day"] = df_dates_dayahead[0].str[6:8]
date_last_dayahead = df_dates_dayahead.sort_values(["Year","Month","Day"], ascending=[False, False, False]).reset_index().iloc[0,1][:8]
dt_last_dayahead = date(int(date_last_dayahead[:4]),int(date_last_dayahead[4:6]),int(date_last_dayahead[6:8]))
list_dates_dayahead = get_dates_between(dt_last_dayahead, dt_today)
list_dates_dayahead

['20260101',
 '20260102',
 '20260103',
 '20260104',
 '20260105',
 '20260106',
 '20260107',
 '20260108',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260109',
 '20260201',
 '20260202',
 '20260203']

In [20]:
#load file setup
url_load = "https://mis.nyiso.com/public/csv/pal/"
file_load = "pal.csv"
path_load = "Inputs\\load\\"
list_load = os.listdir(path_load)
df_dates_load = pd.DataFrame(list_load)
df_dates_load["Year"] = df_dates_load[0].str[:4]
df_dates_load["Month"] = df_dates_load[0].str[4:6]
df_dates_load["Day"] = df_dates_load[0].str[6:8]
date_last_load = df_dates_load.sort_values(["Year","Month","Day"], ascending=[False, False, False]).reset_index().iloc[0,1][:8]
dt_last_load = date(int(date_last_load[:4]),int(date_last_load[4:6]),int(date_last_load[6:8]))
list_dates_load = get_dates_between(dt_last_load, dt_today)
list_dates_load

['20260201', '20260202', '20260203']

In [21]:
#flows file setup
url_flows = "https://mis.nyiso.com/public/csv/ExternalLimitsFlows/"
file_flows = "ExternalLimitsFlows.csv"
path_flows = "Inputs\\flows\\"
list_flows = os.listdir(path_flows)
df_dates_flows = pd.DataFrame(list_flows)
df_dates_flows["Year"] = df_dates_flows[0].str[:4]
df_dates_flows["Month"] = df_dates_flows[0].str[4:6]
df_dates_flows["Day"] = df_dates_flows[0].str[6:8]
date_last_flows = df_dates_flows.sort_values(["Year","Month","Day"], ascending=[False, False, False]).reset_index().iloc[0,1][:8]
dt_last_flows = date(int(date_last_flows[:4]),int(date_last_flows[4:6]),int(date_last_flows[6:8]))
list_dates_flows = get_dates_between(dt_last_flows, dt_today)
list_dates_flows

['20260201', '20260202', '20260203']

In [30]:
#download from nyiso website
#//FIXME - Need to add a functionality to retrieve files older than 1 week, as the NYISO website stores them in zip files

#//STUB - Automatically set to download data that is newer than the last day found in the folder but older than today.
        # Customize if needed. Date format is yyyymmdd (ie: 20250410)

try:
        for day in list_dates_realtime:
                urlretrieve(url_realtime + day + file_realtime, path_realtime + day + file_realtime)

        for day in list_dates_dayahead:
                urlretrieve(url_dayahead + day + file_dayahead, path_dayahead + day + file_dayahead)

        for day in list_dates_load:
                urlretrieve(url_load + day + file_load, path_load + day + file_load)

        for day in list_dates_flows:
                urlretrieve(url_flows + day + file_flows, path_flows + day + file_flows)
except:
        pass

#### Rearrange data

In [31]:
#real-time data

list_files = os.listdir(path_realtime)

df_realtime = pd.DataFrame()
for file in list_files:
    _df_realtime = pd.read_csv(path_realtime+file)
    #concatenate each file to single dataframe
    df_realtime = pd.concat([df_realtime,_df_realtime],ignore_index=True)

df_realtime = df_realtime.drop_duplicates()
df_realtime["Time Stamp"] = pd.to_datetime(df_realtime["Time Stamp"])

df_realtime_s = pd.DataFrame()

#resample at 15min interval, average lbmp
for i in df_realtime["Name"].unique():
    _df = df_realtime[df_realtime["Name"]==i].resample("15min",on="Time Stamp").mean(["LBMP ($/MWHr)"])
    df_ = pd.DataFrame(_df["LBMP ($/MWHr)"])
    df_["Name"] = i
    df_realtime_s = pd.concat([df_realtime_s, df_])

df_realtime_s.head()

,LBMP ($/MWHr),Name
Time Stamp,,
2024-01-01 00:00:00,32.830000,CAPITL
2024-01-01 00:15:00,31.286667,CAPITL
2024-01-01 00:30:00,27.846667,CAPITL
2024-01-01 00:45:00,30.323333,CAPITL
2024-01-01 01:00:00,31.110000,CAPITL


In [32]:
#day-ahead data

list_files = os.listdir(path_dayahead)

df_dayahead = pd.DataFrame()
for file in list_files:
    _df_dayahead = pd.read_csv(path_dayahead+file)
    #concatenate each file to single dataframe
    df_dayahead = pd.concat([df_dayahead,_df_dayahead],ignore_index=True)

df_dayahead = df_dayahead.drop_duplicates()
df_dayahead["Time Stamp"] = pd.to_datetime(df_dayahead["Time Stamp"])

df_dayahead_s = pd.DataFrame()

#no resampling since hourly
# #resample at 15min interval, average lbmp
# for i in df_dayahead["Name"].unique():
#     _df = df_dayahead[df_dayahead["Name"]==i].resample("15min",on="Time Stamp").mean(["LBMP ($/MWHr)"])
#     df_ = pd.DataFrame(_df["LBMP ($/MWHr)"])
#     df_["Name"] = i
#     df_dayahead_s = pd.concat([df_dayahead_s, df_])

df_dayahead_s["LBMP ($/MWHr)"] = df_dayahead["LBMP ($/MWHr)"]
df_dayahead_s["Name"] = df_dayahead["Name"]
df_dayahead_s["Time Stamp"] = df_dayahead["Time Stamp"]
df_dayahead_s = df_dayahead_s.set_index("Time Stamp")
df_dayahead_s.head()

,LBMP ($/MWHr),Name
Time Stamp,,
2024-01-01,25.02,CAPITL
2024-01-01,24.25,CENTRL
2024-01-01,25.10,DUNWOD
2024-01-01,23.59,GENESE
2024-01-01,24.27,H Q


In [33]:
#flows data

list_files = os.listdir(path_flows)

df_flows = pd.DataFrame()
for file in list_files:
    _df_flows = pd.read_csv(path_flows+file)
    #concatenate each file to single dataframe
    df_flows = pd.concat([df_flows,_df_flows],ignore_index=True)

df_flows = df_flows.drop_duplicates()
df_flows["Timestamp"] = pd.to_datetime(df_flows["Timestamp"])

df_flows_s = pd.DataFrame()

#resample at 15min interval, sum mwh
for i in df_flows["Interface Name"].unique():
    _df = df_flows[df_flows["Interface Name"]==i].resample("15min",on="Timestamp").sum()
    df_ = pd.DataFrame(_df["Flow (MWH)"])
    df_["Interface Name"] = i
    df_flows_s = pd.concat([df_flows_s, df_])

df_flows_s.head()

,Flow (MWH),Interface Name
Timestamp,,
2024-01-01 00:00:00,3399.67,CENTRAL EAST - VC
2024-01-01 00:15:00,3677.74,CENTRAL EAST - VC
2024-01-01 00:30:00,3623.82,CENTRAL EAST - VC
2024-01-01 00:45:00,3509.15,CENTRAL EAST - VC
2024-01-01 01:00:00,3469.61,CENTRAL EAST - VC


In [34]:
#load data

list_files = os.listdir(path_load)

df_load = pd.DataFrame()
for file in list_files:
    _df_load = pd.read_csv(path_load+file)
    #concatenate each file to single dataframe
    df_load = pd.concat([df_load,_df_load],ignore_index=True)

df_load = df_load.drop_duplicates()
df_load["Time Stamp"] = pd.to_datetime(df_load["Time Stamp"])

df_load_s = pd.DataFrame()

#resample at 15min interval, average load
for i in df_load["Name"].unique():
    _df = df_load[df_load["Name"]==i].resample("15min",on="Time Stamp").mean(["Load"])
    df_ = pd.DataFrame(_df["Load"])
    df_["Name"] = i
    df_load_s = pd.concat([df_load_s, df_])

df_load_s.head()

,Load,Name
Time Stamp,,
2024-01-01 00:00:00,1179.416567,CAPITL
2024-01-01 00:15:00,1167.123267,CAPITL
2024-01-01 00:30:00,1157.580833,CAPITL
2024-01-01 00:45:00,1154.124533,CAPITL
2024-01-01 01:00:00,1143.162500,CAPITL


#### Graphs

In [35]:
#graph time series

fig_rt = figure(width=width_, height=height_, title="Real-Time Market Price ($)", x_axis_type="datetime", toolbar_location="above")
fig_da = figure(width=width_, height=height_, title="Day-Ahead Market Price ($)", x_range=fig_rt.x_range, x_axis_type="datetime", toolbar_location="above")
fig_l  = figure(width=width_, height=height_, title="Consumed Load (MW)", x_range=fig_rt.x_range, x_axis_type="datetime", toolbar_location="above")
fig_f  = figure(width=width_, height=height_, title="Imported Electricity (MWh)", x_range=fig_rt.x_range, x_axis_type="datetime", toolbar_location="above")

lines_rt = {}
lines_da = {}
lines_l = {}
lines_t = {}

for zone in df_realtime_s["Name"].unique():
    df_zone_rt = df_realtime_s[df_realtime_s["Name"] == zone]
    lines_rt[zone] = fig_rt.line(list(df_zone_rt.index),df_zone_rt["LBMP ($/MWHr)"],
            line_color = color_zone[zone],
            legend_label=labels[zone],
            name=labels[zone])
    lines_rt[zone].visible = True if zone == "N.Y.C." else False

    df_zone_da = df_dayahead_s[df_dayahead_s["Name"] == zone]
    lines_da[zone] = fig_da.line(list(df_zone_da.index),df_zone_da["LBMP ($/MWHr)"],
            line_color = color_zone[zone],
            legend_label=labels[zone],
            name=labels[zone])
    lines_da[zone].visible = True if zone == "N.Y.C." else False
    
    try:
        df_l = df_load_s[df_load_s["Name"] == zone]
        lines_l[zone] = fig_l.line(list(df_l.index),df_l["Load"],
            line_color = color_zone[zone],
            legend_label=labels[zone],
            name=labels[zone])
        lines_l[zone].visible = True if zone == "N.Y.C." else False
    except:
        continue

for interface in df_flows_s["Interface Name"].unique():
    df_f = df_flows_s[df_flows_s["Interface Name"] == interface]
    lines_t[interface] = fig_f.line(list(df_f.index),df_f["Flow (MWH)"],
            line_color = color_interface[interface],
            legend_label=labels[interface],
            name=labels[interface])
    lines_t[interface].visible = False if interface == "?" else True

fig_rt.legend.location = "left"
fig_rt.legend.click_policy="hide"
fig_da.legend.location = "left"
fig_da.legend.click_policy="hide"
fig_l.legend.location = "left"
fig_l.legend.click_policy="hide"
fig_f.legend.location = "left"
fig_f.legend.click_policy="hide"

# fig_da.vspan(
#     x=[datetime(2025, 1, 21, 0, 0, 0, 0)],
#     line_width=[2], line_color="blue",
# )

# fig_da.vspan(
#     x=[datetime(2025, 1, 22, 0, 0, 0, 0)],
#     line_width=[2], line_color="blue",
# )


fig_rt.add_tools(HoverTool(tooltips=[
     ("name","$name"),
    ("date", "$x{%Y-%m-%d %H:%M:%S}"),
    ("price", "$y{$ 0,0.00}")],
    formatters={'$x': 'datetime'},
    point_policy = 'snap_to_data'))

fig_da.add_tools(HoverTool(tooltips=[
     ("name","$name"),
    ("date", "$x{%Y-%m-%d %H:%M:%S}"),
    ("price", "$y{$ 0,0.00}")],
    formatters={'$x': 'datetime'},
    point_policy = 'snap_to_data'))

fig_l.add_tools(HoverTool(tooltips=[
     ("name","$name"),
    ("date", "$x{%Y-%m-%d %H:%M:%S}"),
    ("load", "$y{0,0.0 'MW'}")],
    formatters={'$x': 'datetime'},
    point_policy = 'snap_to_data'))

fig_f.add_tools(HoverTool(tooltips=[
     ("name","$name"),
    ("date", "$x{%Y-%m-%d %H:%M:%S}"),
    ("flow", "$y{0,0 'MWh'}")],
    formatters={'$x': 'datetime'},
    point_policy = 'snap_to_data'))
p = column(fig_rt,fig_da,fig_l,fig_f)

div = Div(
    text="""
        <p>Select the circle's size using this control element:</p>
        """,
    width=200,
    height=30,
)

# range_slider = DatetimeRangeSlider(
#     title="Adjust x-axis range", # a title to display above the slider
#     start=date_first,  # set the minimum value for the slider
#     end=dt_today - timedelta(1),  # set the maximum value for the slider
#     # step=1,  # increments for the slider
#     value=(fig_rt.x_range.start, fig_rt.x_range.end),  # initial values for slider
# )

# range_slider.js_link("value", fig_rt.x_range, "start", attr_selector=0)
# range_slider.js_link("value", fig_rt.x_range, "end", attr_selector=1)

# range_slider.js_on_change("value", CustomJS(code="""
#     console.log('datetime_range_slider: value=' + this.value, this.toString())
# """))

layout = layout(
    [   
        # [range_slider],
        [div],
        [p],
    ],
)

output_file(filename="nyiso_tracker.html", title="NYISO Tracker")
# save(layout)
show(layout)


In [36]:
output_notebook()
q = column(fig_rt,fig_da,fig_l,fig_f)
show(q)

Loading BokehJS ...

RuntimeError: Models must be owned by only a single document, Line(id='p1743', ...) is already in a doc

In [ ]:
#key metrics
